In [2]:
import pandas as pd
import numpy as np
import copy
from sklearn.impute import KNNImputer

In [3]:
data = pd.read_csv("/Users/bianhanzhang/Desktop/data/SP500.csv")
data.head(5)

,date,MON,MAR,NOV,A,AAL,AAP,AAPL,ABBV,ABC,...,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
0,2013-02-08,101.15,40.97,69.08,45.08,14.75,78.90,67.8542,36.25,46.89,...,28.24,37.51,88.61,42.87,31.84,27.09,65.30,75.85,24.14,33.05
1,2013-02-11,102.12,41.22,68.26,44.60,14.46,78.39,68.5614,35.85,46.76,...,28.31,37.46,88.28,42.84,31.96,27.46,64.55,75.65,24.21,33.26
2,2013-02-12,101.77,41.39,68.92,44.62,14.27,78.60,66.8428,35.42,46.96,...,28.41,37.58,88.46,42.87,31.84,27.95,64.75,75.44,24.49,33.74
3,2013-02-13,102.81,41.18,68.96,44.75,14.66,78.97,66.7156,35.27,46.64,...,28.42,37.80,88.67,43.08,32.00,28.26,64.41,76.00,24.74,33.55
4,2013-02-14,102.00,41.16,70.29,44.58,13.99,78.84,66.6556,36.57,46.77,...,28.22,38.44,88.52,42.91,32.12,28.47,63.89,76.34,24.63,33.27


In [22]:
df = pd.DataFrame(data)
companies = df.columns

In [16]:
#function to compute asset returns 
def compute_return(price):
    price = np.asarray(price)
    [rows,cols] = price.shape
    rtn = np.zeros([rows-1, cols])
    for j in range(cols):        # j: Assets
        for i in range(rows-1):     # i: Daily Prices
            rtn[i,j]=((price[i+1, j]-price[i,j])/price[i,j])
    return rtn

In [8]:
def knn(data,n_neighbers):
    knn_imputer = KNNImputer(n_neighbors = n_neighbers)
    data_imputed = knn_imputer.fit_transform(data)
    return data_imputed

### 30 Assets Pool

In [5]:
# 3 asset pools:(1) 30 assets (2) 100 assets (3) 400 assets
# df30 pool are based on iShares Core S&P 500 ETF holdings of 2015

#company30_2023 = ["AAPL","MSFT","AMZN","NVDA","GOOGL","TSLA","META","GOOG","BRK.B","XOM","UNH","LLY","JPM","JNJ","V","PG"
#             ,"AVGO","MA","HD","CVX","ABBV","MRK","COST","PEP","ADBE","WMT","CSCO","KO","CRM","TMO"]

company30_2015 = ["AAPL","MSFT","XOM","JNJ","GE","WFC","BRKB","JPM","PG","PFE","CVX","VZ","T","DIS","MRK","FB","BAC"
                  ,"AMZN","C","IBM","GOOGL","KO","GOOG","INTC","GILD","CSCO","CMCSA","ORCL","HD","PEP","V","PM"]
df30 = pd.DataFrame()
for x in company30_2015:
    if x in companies:
        df30[x] = df[x]
print("shape of df30:",df30.shape)
df30.head(5)

shape of df30: (1259, 30)


,AAPL,MSFT,XOM,JNJ,GE,WFC,JPM,PG,PFE,CVX,...,KO,INTC,GILD,CSCO,CMCSA,ORCL,HD,PEP,V,PM
0,67.8542,27.55,88.61,75.48,22.50,34.88,48.63,75.75,26.88,115.64,...,38.77,21.00,40.90,21.16,19.375,34.90,67.01,72.60,39.4500,90.45
1,68.5614,27.86,88.28,75.41,22.45,35.26,48.66,75.81,27.14,115.64,...,38.61,21.03,40.40,21.27,19.320,34.96,66.38,72.36,39.0825,90.35
2,66.8428,27.88,88.46,75.80,22.58,35.51,49.14,75.98,26.99,116.50,...,37.56,21.19,40.83,20.97,19.485,35.11,67.32,72.17,39.2000,90.14
3,66.7156,28.03,88.67,75.66,23.39,35.13,48.68,76.56,27.00,115.53,...,37.21,21.25,41.40,21.14,20.065,34.99,67.45,71.50,38.6925,90.99
4,66.6556,28.04,88.52,75.81,23.41,35.21,49.22,76.78,27.06,115.71,...,36.84,21.23,41.60,20.99,20.170,34.90,67.34,72.28,38.9950,90.01


In [21]:
# impute 30 assets pool
stock_rtn30 = compute_return(df30)

knn_imputer = KNNImputer(n_neighbors = 2)
data_imputed = knn_imputer.fit_transform(stock_rtn30[:,25:26])
stock_rtn30[:,25:26] = data_imputed
print("Shape of array30:",stock_rtn30.shape)
# print(stock_rtn30[0:5,])

Shape of array30: (1258, 30)


In [20]:
# calculate mean return and covariance of the return
# axis = 0 means calculate by each row of a column
mean_return = np.mean(stock_rtn30,axis = 0) * 252
cov_return = np.cov(stock_rtn30,rowvar = False) * 252
print("Shape of mean_rtn:",mean_return.shape)
print("Shape of cov_rtn:",cov_return.shape)
mean_return

Shape of mean_rtn: (30,)
Shape of cov_rtn: (30, 30)


array([ 0.19812157,  0.26179628, -0.01296614,  0.12132151, -0.05961942,
        0.11862429,  0.18948259,  0.02558454,  0.06743116,  0.02048818,
        0.04159436,  0.02191627,  0.14735107,  0.08453622,  0.41842626,
        0.22709959,  0.37989881,  0.14144881, -0.03636228,  0.22177659,
        0.03784733,  0.17800397,  0.18085148,  0.15136476,  0.16476973,
        0.08316929,  0.22604143,  0.09845281,  0.24276759,  0.03125455])

### 100 Assets Pool

In [43]:
# 100 assets pool: randomly select from all companies
company100 = np.random.choice(companies, size = 100, replace = False)
df100 = pd.DataFrame(dtype="int")
for x in company100:
    df100[x] = df[x]
print("shape of df100",df100.shape)
print(df100.columns)
df100.head(5)

shape of df100 (1259, 100)
Index(['MO', 'BLL', 'NTAP', 'AFL', 'AAP', 'SRE', 'MGM', 'DHI', 'AON', 'AET',
       'NCLH', 'NWL', 'BLK', 'HES', 'NTRS', 'AEP', 'FL', 'AOS', 'OMC', 'PKI',
       'PXD', 'FDX', 'GWW', 'WEC', 'KO', 'SRCL', 'IDXX', 'HST', 'EXPE', 'ADSK',
       'DHR', 'HCA', 'TAP', 'MAT', 'DE', 'MET', 'EOG', 'WYNN', 'GLW', 'WY',
       'HCP', 'SCG', 'MDLZ', 'CNC', 'FRT', 'CSCO', 'CTL', 'APH', 'COST',
       'ORCL', 'DRE', 'ROK', 'FCX', 'XLNX', 'STX', 'ZBH', 'WYN', 'SWKS', 'CCI',
       'CMG', 'OKE', 'CHK', 'UHS', 'AMP', 'SO', 'ALXN', 'RCL', 'FITB', 'UTX',
       'MOS', 'WDC', 'COL', 'LUV', 'APD', 'OXY', 'PSX', 'LMT', 'VAR', 'DISCK',
       'EXC', 'PRU', 'RRC', 'DUK', 'CHRW', 'EBAY', 'UAA', 'JPM', 'ADM', 'DGX',
       'DVA', 'LH', 'INCY', 'ALB', 'TROW', 'PFE', 'HP', 'BEN', 'EA', 'NSC',
       'ECL'],
      dtype='object')


,MO,BLL,NTAP,AFL,AAP,SRE,MGM,DHI,AON,AET,...,LH,INCY,ALB,TROW,PFE,HP,BEN,EA,NSC,ECL
0,34.69,22.680,35.75,50.35,78.90,76.14,13.51,23.08,56.53,50.60,...,90.24,18.32,62.66,73.27,26.88,64.39,46.7333,17.37,69.00,74.40
1,34.75,22.570,35.36,50.26,78.39,75.76,13.05,22.92,56.66,50.38,...,90.06,18.36,62.76,73.13,27.14,64.86,46.7200,17.41,70.57,74.06
2,34.66,22.445,35.41,49.50,78.60,75.83,13.32,24.05,56.70,50.15,...,89.41,18.42,62.70,73.75,26.99,65.42,47.4166,17.54,71.64,74.46
3,34.78,22.600,35.82,48.65,78.97,76.33,13.34,23.83,57.28,49.51,...,89.70,18.65,66.15,74.21,27.00,65.68,47.5166,17.17,71.88,74.67
4,34.11,22.605,35.86,48.83,78.84,75.35,13.29,23.78,57.30,49.56,...,89.19,20.56,65.41,73.84,27.06,67.86,48.1433,17.15,71.97,74.82


In [44]:
# impute 100 assets pool
stock_rtn100 = compute_return(df100)
stock_rtn100 = knn(stock_rtn100,2)
print("Shape of stock_rtn100",stock_rtn100.shape)

Shape of stock_rtn100 (1258, 100)


In [45]:
print(stock_rtn100[0:5,0:3])

[[ 0.00172961 -0.00485009 -0.01090909]
 [-0.00258993 -0.00553833  0.00141403]
 [ 0.0034622   0.00690577  0.01157865]
 [-0.01926394  0.00022124  0.00111669]
 [ 0.00791557  0.00729927  0.01924149]]


In [42]:
mean_return100 = np.mean(stock_rtn100,axis = 0).reshape(-1,1)
cov_return100 = np.cov(stock_rtn100,rowvar = False)
print("Shape of mean_rtn:",mean_return100.shape)
print("Shape of cov_rtn:",cov_return100.shape)
mean_rtn = (1 + mean_return100)**252 -1
# mean_rtn = mean_rtn[0]
print(mean_rtn)

Shape of mean_rtn: (100, 1)
Shape of cov_rtn: (100, 100)
[[ 0.33202788]
 [-0.00680357]
 [ 0.26627419]
 [ 0.07673883]
 [ 0.03719902]
 [ 0.05697536]
 [ 0.28927379]
 [ 0.16809047]
 [ 0.08940692]
 [ 0.2014002 ]
 [ 0.08193993]
 [ 0.11820348]
 [ 0.18849966]
 [ 0.27462336]
 [ 0.10897983]
 [ 0.16995144]
 [ 0.45183117]
 [ 0.04573111]
 [ 0.0695224 ]
 [ 0.19064992]
 [-0.02117617]
 [ 0.01968472]
 [ 0.09180288]
 [ 0.37359269]
 [ 0.17313794]
 [ 0.20853803]
 [ 0.23317344]
 [ 0.06634662]
 [ 0.26678139]
 [ 0.1556723 ]
 [ 0.18820987]
 [ 0.06592273]
 [ 0.21134768]
 [-0.05479026]
 [ 0.0551374 ]
 [ 0.03214435]
 [ 0.29908528]
 [ 0.15969193]
 [ 0.12627768]
 [ 0.24448084]
 [ 0.16001087]
 [-0.11801231]
 [ 0.08008697]
 [ 0.21398362]
 [ 0.12712821]
 [ 0.2969625 ]
 [ 0.16552065]
 [ 0.12625098]
 [ 0.1787278 ]
 [ 0.34957389]
 [ 0.13938149]
 [ 0.09657712]
 [ 0.51292667]
 [ 0.17905808]
 [-0.08750405]
 [ 0.1933617 ]
 [ 0.12148846]
 [ 0.15183671]
 [ 0.10866442]
 [ 0.13523947]
 [ 0.36703587]
 [ 0.15688859]
 [ 0.34656145

In [46]:
SP500_100_assets_pool = np.concatenate((mean_return100,cov_return100),axis = 1)

file_name = "SP500_100_assets_pool2.csv"
delimiter=","
col_names = copy.copy(df100.columns)
col_names = col_names.insert(0, "mean_return")

np.savetxt(file_name, SP500_100_assets_pool, delimiter=delimiter, header = ",".join(col_names), comments="")